In [1]:
%pip install --quiet --upgrade pip 
%pip install numpy --quiet
%pip install Pandas --quiet
%pip install sklearn --quiet
%pip install ipywidgets --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Horse Racing Results Analysis using Linear regression #

Use simple linear regression techniques to determine if we can reliably predict the **speed** of a horse in a given race based on its previous performance. 

Data used here is derived from the features extracted by the [Feature Analysis](https://github.com/LeeSanderson/RacingData/blob/main/Data/FeatureAnalysis.ipynb) notebook.

In [2]:
import numpy as np
import pandas as pd
import math
from abc import ABC, abstractmethod
from datetime import datetime, date

In [3]:
races = pd.read_csv("Race_Features.csv")
races['Off'] =  pd.to_datetime(races['Off'], format='%Y-%m-%d %H:%M:%S')
races.columns

Index(['RaceId', 'CourseId', 'RaceType', 'Off', 'DecimalOdds',
       'OfficialRating', 'RacingPostRating', 'TopSpeedRating',
       'DistanceInMeters', 'Going', 'Surface', 'HorseId', 'HorseName',
       'JockeyId', 'JockeyName', 'Age', 'HeadGear', 'RaceCardNumber',
       'StallNumber', 'WeightInPounds', 'FinishingPosition',
       'OverallBeatenDistance', 'RaceTimeInSeconds', 'Wins',
       'Surface_AllWeather', 'Surface_Dirt', 'Surface_Turf', 'Going_Firm',
       'Going_Good', 'Going_Good_To_Firm', 'Going_Good_To_Soft', 'Going_Heavy',
       'Going_Soft', 'RaceType_Flat', 'RaceType_Hurdle', 'RaceType_Other',
       'RaceType_SteepleChase', 'Speed', 'HorseCount', 'KnownHorseAndJockey',
       'NumberOfPriorRaces', 'LastRaceGoing', 'LastRaceSurface',
       'LastRaceDistanceInMeters', 'LastRaceWeightInPounds', 'LastRaceSpeed',
       'DaysRested', 'LastRaceDecimalOdds', 'LastRaceOfficialRating',
       'LastRaceRacingPostRating', 'LastRaceTopSpeedRating',
       'LastRaceAvgRelFinishi

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

race_info = ['RaceId', 'CourseId', 'Off', 'HorseId', 'HorseName', 'HorseCount', 'JockeyId', 'JockeyName', 'DecimalOdds', 'Wins', 'FinishingPosition']
predictors = ([
    'DistanceInMeters', 
    'WeightInPounds', 
    'Surface_AllWeather', 'Surface_Dirt', 'Surface_Turf', 
    'Going_Firm', 'Going_Good', 'Going_Good_To_Firm', 'Going_Good_To_Soft', 'Going_Heavy', 'Going_Soft', 
    'RaceType_Flat', 'RaceType_Hurdle', 'RaceType_Other', 'RaceType_SteepleChase', 

    'LastRaceDistanceInMeters', 
    'LastRaceWeightInPounds', 
    'LastRaceSpeed',
    'DaysRested',
    'LastRaceAvgRelFinishingPosition', 
    'LastRaceSurface_AllWeather', 'LastRaceSurface_Dirt', 'LastRaceSurface_Turf', 
    'LastRaceGoing_Good', 'LastRaceGoing_Good_To_Soft', 'LastRaceGoing_Soft', 'LastRaceGoing_Good_To_Firm', 'LastRaceGoing_Firm', 'LastRaceGoing_Heavy', 
    'LastRaceRaceType_Other', 'LastRaceRaceType_Hurdle', 'LastRaceRaceType_SteepleChase', 'LastRaceRaceType_Flat', 
    
    'JockeyNumberOfPriorRaces',
    'DaysSinceJockeyLastRaced',     
    'JockeyWinPercentage',
    'JockeyTop3Percentage',
    'JockeyAvgRelFinishingPosition'
    ])
prediction = ["Speed"]

train = races[race_info + predictors + prediction].dropna().copy()

In [5]:
# Cap rested days
train.loc[train["DaysRested"] > 10, "DaysRested"] = 10
train.loc[train["DaysSinceJockeyLastRaced"] > 10, "DaysSinceJockeyLastRaced"] = 10

# Cap prior races
# train.loc[train["JockeyNumberOfPriorRaces"] > 400] = 400 (negative effect)

train[predictors].describe()

,DistanceInMeters,WeightInPounds,Surface_AllWeather,Surface_Dirt,Surface_Turf,Going_Firm,Going_Good,Going_Good_To_Firm,Going_Good_To_Soft,Going_Heavy,Going_Soft,RaceType_Flat,RaceType_Hurdle,RaceType_Other,RaceType_SteepleChase,LastRaceDistanceInMeters,LastRaceWeightInPounds,LastRaceSpeed,DaysRested,LastRaceAvgRelFinishingPosition,LastRaceSurface_AllWeather,LastRaceSurface_Dirt,LastRaceSurface_Turf,LastRaceGoing_Good,LastRaceGoing_Good_To_Soft,LastRaceGoing_Soft,LastRaceGoing_Good_To_Firm,LastRaceGoing_Firm,LastRaceGoing_Heavy,LastRaceRaceType_Other,LastRaceRaceType_Hurdle,LastRaceRaceType_SteepleChase,LastRaceRaceType_Flat,JockeyNumberOfPriorRaces,DaysSinceJockeyLastRaced,JockeyWinPercentage,JockeyTop3Percentage,JockeyAvgRelFinishingPosition
count,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000,287297.000000
mean,2272.222613,153.928269,0.236449,0.018347,0.745159,0.027686,0.464509,0.094122,0.224444,0.068295,0.120736,0.006965,0.185313,0.729096,0.078626,2236.471703,153.793625,15.120765,9.764432,0.529876,0.232693,0.017915,0.749312,0.462876,0.222609,0.120694,0.098285,0.026857,0.068473,0.731410,0.183939,0.072970,0.011681,404.313132,3.037153,0.110636,0.327848,0.551121
std,1146.052475,15.898622,0.424902,0.134203,0.435772,0.164071,0.498740,0.291999,0.417216,0.252252,0.325820,0.083165,0.388552,0.444428,0.269155,1125.161402,15.821767,1.527463,0.992283,0.192947,0.422549,0.132644,0.433410,0.498621,0.415999,0.325772,0.297700,0.161666,0.252556,0.443227,0.387435,0.260087,0.107447,400.587328,2.799772,0.063799,0.124655,0.070000
min,804.000000,119.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,804.000000,119.000000,9.763058,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.052632
25%,1407.000000,144.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1407.000000,144.000000,13.808751,10.000000,0.402207,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,103.000000,1.000000,0.071429,0.245179,0.511164
50%,1809.000000,149.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1709.000000,149.000000,15.555087,10.000000,0.519231,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,277.000000,2.000000,0.103641,0.324201,0.546858
75%,3217.000000,165.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3217.000000,164.000000,16.271351,10.000000,0.648485,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,578.000000,4.000000,0.142857,0.405738,0.589497
max,6937.000000,200.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6937.000000,199.000000,19.670069,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2441.000000,10.000000,1.000000,1.000000,1.000000


In [6]:
# Now that we've dropped any rows with na values, calculate which races in which we can predict the speed for all horses.
train = train.drop("PredictableHorseCount", axis=1, errors='ignore')
groups = train.groupby(['RaceId']).apply(lambda g: pd.Series({'PredictableHorseCount': g['RaceId'].count()}, index=['PredictableHorseCount']))
train = pd.merge(train, groups, how='left', on=['RaceId'])

# 50% of races have 11 horse, 25% have 8 or less, 14% have 6 or less, 9% have 5 or less. Fewer horses should be more predictable
all_races_count = len(races["RaceId"].unique())
all_predictable = train[(train["HorseCount"] == train["PredictableHorseCount"]) & (train["HorseCount"] <= 5)]["RaceId"].unique().tolist()
all_predictable_count = len(all_predictable)
print(f"Possible predictable races = {all_predictable_count} out of {all_races_count} ({all_predictable_count * 100 / all_races_count}%)")

Possible predictable races = 4033 out of 37890 (10.643969385062022%)


In [7]:
from sklearn.model_selection import train_test_split

_, test_race_ids = train_test_split(all_predictable, test_size=0.2, random_state = 42)

test = train[train["RaceId"].isin(test_race_ids)]
train = train[train["RaceId"].isin(test_race_ids) == False]

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

# Create a pipeline that includes feature scaling and linear regression
model = make_pipeline(StandardScaler(), PolynomialFeatures(degree=2, interaction_only=False), LinearRegression())

In [9]:
#help(model.fit)

In [10]:
inputs = train[predictors]
targets = train[prediction[0]].values

# print(inputs.shape)
# print(targets.shape)

In [11]:
model.fit(inputs, targets)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [12]:
train_predictions = model.predict(inputs)
train_mse = mean_squared_error(targets, train_predictions)
train_rmse = np.sqrt(train_mse)
train_rmse

0.47951064805895616

In [13]:
test_inputs = test[predictors]
predictions = model.predict(test_inputs)
test_with_predictions = test.copy()
test_with_predictions["PredictedSpeed"] = predictions

test_mse = mean_squared_error(test_with_predictions["Speed"], predictions)
test_rmse = np.sqrt(test_mse)
test_rmse

0.47228779301963336

In [23]:
from sklearn.model_selection import cross_val_score
num_folds = 5

# Perform k-fold cross-validation
scores = cross_val_score(model, inputs, targets, cv=num_folds, scoring="neg_mean_squared_error")

rmse_scores = np.sqrt(-scores)

# Print the RMSE scores for each fold
for i, rmse in enumerate(rmse_scores):
    print(f"Fold {i + 1} RMSE: {rmse:.2f}")

# Calculate the mean and standard deviation of RMSE scores
mean_rmse = rmse_scores.mean()
std_rmse = rmse_scores.std()

print(f"Mean speed: {targets.mean()}")
print(f"Std speed: {targets.std()}")
print(f"Mean RMSE: {mean_rmse:.2f}")
print(f"Std RMSE: {std_rmse:.2f}")

# Large values of cross-validation errors indicate multi-co-linearity

Fold 1 RMSE: 4674973871.12
Fold 2 RMSE: 4472654395.62
Fold 3 RMSE: 862192281.50
Fold 4 RMSE: 3882891445.90
Fold 5 RMSE: 962805254.13
Mean speed: 15.116678155607909
Mean RMSE: 2971103449.66
Std RMSE: 1701173416.43


In [21]:
scores / len(targets)

array([-3.09024858e+13, -2.89680748e+13, -1.39066617e+12, -1.76527259e+13,
       -1.98320519e+12])

In [14]:
test_with_predictions["PredictedRank"] = test_with_predictions.groupby("RaceId")["PredictedSpeed"].rank(method="dense", ascending=False)

In [15]:
(test_with_predictions[test_with_predictions["RaceId"] == 789029]
 [["RaceId", "HorseId", "HorseName", "FinishingPosition", "PredictedRank", "Speed", "PredictedSpeed", "Speed", "HorseCount", "DecimalOdds"]]
 #.sort_values("PredictedRank")
 .head(200))

,RaceId,HorseId,HorseName,FinishingPosition,PredictedRank,Speed,PredictedSpeed,Speed,HorseCount,DecimalOdds
1183,789029,2951763,Chalk Stream,1,2.0,15.562294,15.003387,15.562294,5,2.2
1184,789029,3142176,Barn Owl,2,3.0,15.506711,14.984802,15.506711,5,3.5
1185,789029,3481885,Carrigillihy,3,1.0,15.457025,15.020111,15.457025,5,4.5
1186,789029,3098714,Grantley,4,5.0,15.293678,14.684204,15.293678,5,15.0
1187,789029,3098690,Chase The Dollar,5,4.0,15.144306,14.848251,15.144306,5,15.0


In [16]:
correct_winners = test_with_predictions[(test_with_predictions["FinishingPosition"] == 1) & (test_with_predictions["PredictedRank"] == 1)]

correct_winners_count = len(correct_winners)
incorrect_winners_count = len(test_with_predictions[(test_with_predictions["FinishingPosition"] == 1) & (test_with_predictions["PredictedRank"] != 1)])

winnings = correct_winners["DecimalOdds"].sum()

print(f"Correct winners {correct_winners_count}, Incorrect winners {incorrect_winners_count}")
print(f"Win percentage {100.0 * correct_winners_count / incorrect_winners_count}")
print(f"Winnings {winnings}, losses {incorrect_winners_count}, diff  {winnings - incorrect_winners_count}")

Correct winners 274, Incorrect winners 535
Win percentage 51.21495327102804
Winnings 903.8329503829503, losses 535, diff  368.83295038295034


In [17]:
# Base = 23.3630494927759
# PolynomialFeatures(degree=2, interaction_only=True) = 25.01557632398754
# With PolynomialFeatures(degree=2, interaction_only=False) = 25.13252260679763
# With rest days capped =  25.210608424336975
# with max 11 horses in race = 28.612597776862906 (£467)
# with max 8 horses in race = 31.598513011152416 (£336)
# with max 6 horses in race = 38.113207547169814 (£343)
# with max 5 horses in race = 47.77777777777778 (£269)